In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import pickle
import json
import ast
import astor
from fuzzywuzzy import fuzz
from tqdm import tqdm
path = "/home/ubuntu/mydata/pkl_data/distributable/level4_inter"
dic = {}
with open(f"{path}/train.pkl", "rb") as f:
    dic["train"] = pickle.load(f)
with open(f"{path}/dev.pkl", "rb") as f:
    dic["dev"] = pickle.load(f)
with open(f"{path}/test.pkl", "rb") as f:
    dic["test"] = pickle.load(f)


In [ ]:
def get_name(folder):
    prefix = "/home/ubuntu/mydata/virtual/env_"
    assert folder.startswith(prefix)
    return folder[len(prefix):].split("/")[0]

tot = {}
for k, lis in dic.items():
    for i, x in enumerate(lis):
        folder = x["folder"]
        pj = get_name(folder)
        if pj not in tot:
            tot[pj] = []
        tot[pj].append(x)
print(len(tot))

In [ ]:
from collections import Counter
import numpy as np
%matplotlib inline
import math
import matplotlib.pyplot as plt
cnt = Counter()
method = {}
for pj in tot:
    for x in tot[pj]:
        name = x["name"]
        defi = x["definition"][0]
        defi = (defi["uri"], defi["range"]["start"]["line"], defi["range"]["start"]["character"])
        if method.get((name, defi)) is None:
            method[(name, defi)] = Counter()
        method[(name, defi)][pj]+=1


for x in method:
    avg = np.mean([method[x][pj] for pj in method[x]])
    cnt[math.ceil(avg)]+=1
print(cnt)

        
        

In [ ]:
def lastn(text, num=-10):
    lis = text.split("\n")[num:]
    return '\n'.join(lis)

def check(x):
    x = ", ".join([y.strip() for y in x.split(",")])
    x = "(".join([y.strip() for y in x.split("(")])
    x = ")".join([y.strip() for y in x.split(")")])
    x = x.replace("\n", " ")
    x = " ".join(x.split())        
    return x

def sim(a, b):
    x = set(a.split())
    y = set(b.split())
    return len(x&y)

cnt = Counter()
s = Counter()
s1 = 0 
s11 = 0
for pj in tqdm(tot):
    method = {}
    for x in tot[pj]:
        name = x["name"]
        defi = x["definition"][0]
        defi = (defi["uri"], defi["range"]["start"]["line"], defi["range"]["start"]["character"])
        if method.get((name, defi)) is None:
            method[(name, defi)] = []
        method[(name, defi)].append(x)
    for x in method:
        for i, y in enumerate(method[x]):
            yy = lastn(y["context"][0])
            lis = []
            for j, z in enumerate(method[x]):
                if i==j: continue
                assert z["folder"] == y["folder"]
                if z["path"] != y["path"] or (z["position"]["line"]<y["position"]["line"] and \
                                              y["context_position"]["line"]!=z["context_position"]["line"]):
                    try:
                        assert not(z["path"] == y["path"] and y["context_position"]["line"]<z["context_position"]["line"] and y["context_position"]["end_line"]>=z["context_position"]["line"])
                    except:
                        print(y["context_position"])
                        print(z["context_position"])
                        assert False
                    
                    lis.append(z)
            if len(lis)>100:
                import random
                random.seed(42)
                random.shuffle(lis)
                lis = lis[:100]
            lis = [(sim(yy, lastn(z["context"][0])), z) for z in lis]
            lis = sorted(lis, reverse=True, key=lambda z:z[0])
            y["usages"] = [z["context"] + (z["target"],) for _, z in lis]
            
            ff = 10
            for i,z in enumerate(lis[:10]):
                if check(z[1]["target"])==check(y["target"]):
                    ff = i
                    break
            for i in range(ff, 10):
                s[i]+=1
                
            if len(lis)==1:
                if check(lis[0][1]["target"])==check(y["target"]):
                    s11 += 1
                s1+=1
            cnt[len(y["usages"])]+=1

print(s)
print(s1,s11)

In [ ]:
mx = 20
s = [0 for i in range(mx)]
tot = 0
for x in dic["train"]:
    ff = mx
    if len(x["usages"])>0:
        tot+=1
    for i,z in enumerate(x["usages"][:mx]):
        if check(z[-1])==check(x["target"]):
            ff = i
            break
    for i in range(ff, mx):
        s[i]+=1
plt.plot(list(range(1,mx+1)), np.array(s)/tot)

In [ ]:
from training.utils import make_sure_path_exists
make_sure_path_exists(f"{path}_refer/")
for k, lis in dic.items():
    with open(f"{path}_refer/{k}.pkl", "wb") as f:
        pickle.dump(lis, f)

In [ ]:
lis = [min(len(x["usages"]), 49) for x in dic["train"]]
print(len(lis))
plt.hist(lis, range=(0,50), bins=50)

In [ ]:
lis = [x for x in dic["train"] if len(x["usages"])>50]
print(lis[0]["name"])
print(lis[0]["target"])
for y in lis[0]["usages"][:2]:
    print(y)